In [4]:
import pandas as pd

In [5]:
df = pd.read_excel("Abilities.xlsx")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90792 entries, 0 to 90791
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   O*NET-SOC Code      90792 non-null  object 
 1   Title               90792 non-null  object 
 2   Element ID          90792 non-null  object 
 3   Element Name        90792 non-null  object 
 4   Scale ID            90792 non-null  object 
 5   Scale Name          90792 non-null  object 
 6   Data Value          90792 non-null  float64
 7   N                   88608 non-null  float64
 8   Standard Error      88608 non-null  float64
 9   Lower CI Bound      88608 non-null  float64
 10  Upper CI Bound      88608 non-null  float64
 11  Recommend Suppress  90792 non-null  object 
 12  Not Relevant        45396 non-null  object 
 13  Date                90792 non-null  object 
 14  Domain Source       90792 non-null  object 
dtypes: float64(5), object(10)
memory usage: 10.4+ MB


In [7]:
df_importance = df[df["Scale Name"] == "Importance"]

In [8]:
df_level = df[df["Scale Name"] == "Level"]

In [9]:
df_importance.loc[df_importance["Title"] == "Chief Executives", "Element Name"].count()

52

In [10]:
from numpy import True_
df_importance.drop(columns=["N", "Standard Error", "Lower CI Bound", "Upper CI Bound",
                            "Recommend Suppress", "Not Relevant", "Date", "Domain Source"], inplace = True)
df_level.drop(columns=["N", "Standard Error", "Lower CI Bound", "Upper CI Bound",
                            "Recommend Suppress", "Not Relevant", "Date", "Domain Source"], inplace = True)

<ipython-input-10-b49fb6ac6e6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_importance.drop(columns=["N", "Standard Error", "Lower CI Bound", "Upper CI Bound",
<ipython-input-10-b49fb6ac6e6b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level.drop(columns=["N", "Standard Error", "Lower CI Bound", "Upper CI Bound",


In [11]:
df_importance.head()

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value
0,11-1011.00,Chief Executives,1.A.1.a.1,Oral Comprehension,IM,Importance,4.62
2,11-1011.00,Chief Executives,1.A.1.a.2,Written Comprehension,IM,Importance,4.25
4,11-1011.00,Chief Executives,1.A.1.a.3,Oral Expression,IM,Importance,4.50
6,11-1011.00,Chief Executives,1.A.1.a.4,Written Expression,IM,Importance,4.12
8,11-1011.00,Chief Executives,1.A.1.b.1,Fluency of Ideas,IM,Importance,3.88


In [12]:
importance = pd.crosstab(df_importance["Title"], df_importance["Element Name"],values=df_importance["Data Value"], aggfunc='mean').round(1)

In [13]:
level = pd.crosstab(df_level["Title"], df_level["Element Name"],values=df_level["Data Value"], aggfunc='mean').round(1)

In [14]:
print(importance.shape)
print(level.shape)

(873, 52)
(873, 52)


We have 873 occupations with 52 abilities (mapped by importance and proficiency level)

In [15]:
merged_df = importance.merge(level, left_on="Title", right_on="Title",
                             suffixes = ("_importance","_level"))

In [16]:
merged_df.reset_index(inplace=True)

In [17]:
job_threat_index = pd.read_csv("My_Data.csv")

In [18]:
job_threat_index.head()

,Job titiles,AI Impact,Tasks,AI models,AI_Workload_Ratio,Domain
0,Communications Manager,98%,365,2546,0.143362,Communication & PR
1,Data Collector,95%,299,2148,0.139199,Data & IT
2,Data Entry,95%,325,2278,0.142669,Administrative & Clerical
3,Mail Clerk,95%,193,1366,0.141288,Leadership & Strategy
4,Compliance Officer,92%,194,1369,0.141709,Medical & Healthcare


In [19]:
job_threat_index.shape

(4706, 6)

In [20]:
merged_df.columns

Index(['Title', 'Arm-Hand Steadiness_importance',
       'Auditory Attention_importance', 'Category Flexibility_importance',
       'Control Precision_importance', 'Deductive Reasoning_importance',
       'Depth Perception_importance', 'Dynamic Flexibility_importance',
       'Dynamic Strength_importance', 'Explosive Strength_importance',
       ...
       'Speed of Limb Movement_level', 'Stamina_level',
       'Static Strength_level', 'Time Sharing_level', 'Trunk Strength_level',
       'Visual Color Discrimination_level', 'Visualization_level',
       'Wrist-Finger Speed_level', 'Written Comprehension_level',
       'Written Expression_level'],
      dtype='object', name='Element Name', length=105)

In [62]:
job_threat_index.loc[job_threat_index["Job titiles"]=="*Postsecondary*"]

,Job titiles,AI Impact,Tasks,AI models,AI_Workload_Ratio,Domain


In [22]:
import re

In [23]:
merged_df_split = [re.split(r',|, and', x) for x in merged_df["Title"]]

In [24]:
for x in merged_df_split:
  for y in range(len(x)):
    if x[y][-1] == "s":
      x[y] = x[y].rstrip(x[y][-1])

In [25]:
for x in merged_df_split:
  for y in range(len(x)):
    if x[y][-2:] == "ie":
      x[y] = re.sub(r'.{2}$',"y",x[y])

In [30]:
for x in merged_df_split:
  for y in range(len(x)):
      x[y] = x[y].replace('s and',"")

In [33]:
for x in merged_df_split:
  for y in range(len(x)):
    if x[y][0:5] == " and ":
      x[y] = re.sub(r'^.{0,5}', "", x[y])

In [41]:
for x in merged_df_split:
  for y in range(len(x)):
    if x[y][0] == " ":
      x[y] = x[y].strip()

In [58]:
exceptions = []
for x in merged_df_split:
  for y in range(len(x)):
    if "except" in x[y]:
      m = y
      for y in range(m,len(x)+1):
        exceptions.append(x[y])

In [59]:
merged_df_split

[['Accountant Auditor'],
 ['Actor'],
 ['Actuary'],
 ['Acupuncturist'],
 ['Acute Care Nurse'],
 ['Adapted Physical Education Specialist'],
 ['Adhesive Bonding Machine Operator Tender'],
 ['Administrative Law Judge', 'Adjudicator', 'Hearing Officer'],
 ['Administrative Services Manager'],
 ['Adult Basic Education',
  'Adult Secondary Education',
  'English as a Second Language Instructor'],
 ['Advanced Practice Psychiatric Nurse'],
 ['Advertising Sales Agent'],
 ['Advertising and Promotions Manager'],
 ['Aerospace Engineering and Operations Technologist Technician'],
 ['Aerospace Engineer'],
 ['Agent Business Managers of Artist', 'Performer', 'Athlete'],
 ['Agricultural Engineer'],
 ['Agricultural Equipment Operator'],
 ['Agricultural Inspector'],
 ['Agricultural Sciences Teacher', 'Postsecondary'],
 ['Agricultural Technician'],
 ['Air Traffic Controller'],
 ['Aircraft Cargo Handling Supervisor'],
 ['Aircraft Mechanic Service Technician'],
 ['Aircraft Structure', 'Surface', 'Rigging', 'S

In [43]:
merged_df_split_series = pd.Series([merged_df_split])

In [44]:
df = merged_df_split_series.apply(pd.Series).stack().reset_index(drop=True)

In [48]:
merged_df_split

[['Accountant Auditor'],
 ['Actor'],
 ['Actuary'],
 ['Acupuncturist'],
 ['Acute Care Nurse'],
 ['Adapted Physical Education Specialist'],
 ['Adhesive Bonding Machine Operator Tender'],
 ['Administrative Law Judge', 'Adjudicator', 'Hearing Officer'],
 ['Administrative Services Manager'],
 ['Adult Basic Education',
  'Adult Secondary Education',
  'English as a Second Language Instructor'],
 ['Advanced Practice Psychiatric Nurse'],
 ['Advertising Sales Agent'],
 ['Advertising and Promotions Manager'],
 ['Aerospace Engineering and Operations Technologist Technician'],
 ['Aerospace Engineer'],
 ['Agent Business Managers of Artist', 'Performer', 'Athlete'],
 ['Agricultural Engineer'],
 ['Agricultural Equipment Operator'],
 ['Agricultural Inspector'],
 ['Agricultural Sciences Teacher', 'Postsecondary'],
 ['Agricultural Technician'],
 ['Air Traffic Controller'],
 ['Aircraft Cargo Handling Supervisor'],
 ['Aircraft Mechanic Service Technician'],
 ['Aircraft Structure', 'Surface', 'Rigging', 'S

Split on "," and "and":
1. Have occupations that have "